## Recurrence Neural Networks Notebook

### Description:

This notebook aims to implement a **Recurrence Neural Network** for Forecasting Time Series.

As explained in the **SeriesAnalysis** notebook, this notebook may use the results already obtained in **Series Analysis**, so that the focus can be only on the Model itself. The results that will be used are the stationary series obtained from the original series, in order to verify if the **Recurrence Networks** are capable of forecasting both stationary and non stationary series

### Recurrence Networks brief introduction

# COMPLETE


In [ ]:
#Packages needed

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# COMPLETE WITH PACKAGES FOR RECURRENCE

### Some useful functions -> COMPLETE WITH MORE IF NEEDED AND DELETE ONES NOT NEEDED

In [ ]:
#Read and filter data from csv files, as well as handle missing values
def read_filter(path,index,value):
    df = pd.read_csv(path)
    series = df.loc[:,[index,value]]
    series.set_index(index,inplace=True)
    series.index=pd.to_datetime(series.index)
    series = series.asfreq('d')
    series = series.interpolate()
    return series

#Ploting a single time series
def plot_series(serie,title,xlabel,ylabel):
    plt.figure()
    plt.plot(serie)
    plt.title=title
    plt.xlabel=xlabel
    plt.ylabel=ylabel
    plt.show()
    
    
# differentiate the series
def diff(series,t):
    for i in range(t):
        series = series - series.shift(1)
        series = series.dropna()
    
    return series

#train test split
def train_test(serie,numtest):
    train = serie.iloc[0:len(serie)-numtest]
    test = serie.iloc[len(serie)-numtest:]    
    return train, test
    
#Compute some error metrics
def errors(actual, predicted):
    mape = np.mean(np.abs(predicted - actual)/np.abs(actual))  # MAPE
    me = np.mean(predicted - actual)             # ME
    mae = np.mean(np.abs(predicted - actual))    # MAE
    mpe = np.mean((predicted - actual)/actual)   # MPE
    rmse = np.mean((predicted - actual)**2)**.5  # RMSE
    
    errors = [['MAPE',mape],['ME',me],['MAE',mae],['MPE',mpe],['RMSE',rmse]]
    
    errors = pd.DataFrame(errors,columns=['Error','Value'])
    return errors

### Importing the series and differenciating in order to get stationary behavior

# NOTE:

Feel free to pick just one to implement the code. After the code if ready and good, we can simply repeat for the others.
I would suggest pick one that is "diff".

In [ ]:
#Importing the dataset and making some changes
USD_CAD_serie = read_filter('../data/CAD=X.csv','Date','Adj Close')
USD_BRL_serie = read_filter('../data/BRL=X.csv','Date','Adj Close')
AmericanAirlines_serie = read_filter('../data/AAL2010_2019.csv','Date','Adj Close')

#differentiating the series
USD_CAD_serie_diff1 = diff(USD_CAD_serie,1)
USD_BRL_serie_diff1 = diff(USD_BRL_serie,1)
AmericanAirlines_serie_diff1 = diff(AmericanAirlines_serie,1)

## Training and Test Sets

We must divide our time series into a training and test sample. Since our series has anual seasonality, let's leave the last yeas as test sample and the rest of the series as training.

The size of the test series is the size of the largest forecast horizon.

In [1]:
largest_horizon = 12 #c12 weeks, 3 months

train_American_diff1, test_American_diff1 = train_test(AmericanAirlines_serie_diff1,largest_horizon)
train_USD_CAD_diff1, test_USD_CAD_diff1 = train_test(USD_CAD_serie_diff1,largest_horizon)
train_USD_BRL_diff2, test_USD_BRL_diff1 = train_test(USD_BRL_serie_diff1,largest_horizon)

NameError: name 'train_test' is not defined